# NCAA Tournament Simulations

This notebook simulates the 2015 NCAA Tournament using our distribution of features.

## Setup

In [48]:
# Libraries.
import numpy as np
import pandas as pd
import pickle

# Custom libraries.
import game_predictions
# game_predictions = reload(game_predictions)

# Library settings.
% matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [77]:
# Paths.
# Pre-tournament statistics path.
stats_path = '../../../data/kenpom_summaries/summary15_pt.csv'
# Bracket paths.
tournament_bracket_path = '../../../data/tournament_brackets/2015_64.csv'
final_four_bracket_path = '../../../data/tournament_brackets/2015_4.csv'
# Export paths.
export_base_path = '../../../data/simulations/'

In [91]:
# Function to reformat matchups in a different format.
def reformat_matchups(matchups):
    # Split data.
    data_1 = matchups[matchups.winner < matchups.loser].copy()
    data_2 = matchups[matchups.winner > matchups.loser].copy()
    # Rename columns.
    data_1 = data_1.rename(columns={'winner':'team_1','loser':'team_2','count':'team_1_wins'})
    data_2 = data_2.rename(columns={'winner':'team_2','loser':'team_1','count':'team_2_wins'})
    # Re-aggregate and return.
    data_recombined = data_1.append(data_2).fillna(0).groupby(['team_1','team_2','round_of']).agg(np.sum).reset_index()
    data_recombined.team_1_wins = data_recombined.team_1_wins.astype(np.int)
    data_recombined.team_2_wins = data_recombined.team_2_wins.astype(np.int)
    return data_recombined

## Data

In [3]:
# Read stats.
pre_tournament_stats = pd.read_csv(stats_path)

In [23]:
# Read brackets.
tournament_bracket = pd.read_csv(tournament_bracket_path, header=None).values
final_four_bracket = pd.read_csv(final_four_bracket_path, header=None).values
championship_bracket = np.array([['Wisconsin','Duke']])

In [6]:
# Slice trace.
with open('slice_samples_traces.pkl', 'rb') as fid:
    slice_trace = pickle.load(fid)
# Metropolis-Hastings trace.
with open('MH_traces.pkl', 'rb') as fid:
    mh_trace = pickle.load(fid)

In [7]:
# Features.
features = ['diff_Pythag','diff_AdjOE','diff_AdjDE','location_Away','diff_RankAdjOE']

---

## Deterministic Simulations

### Slice

#### Whole-Tournament

In [80]:
# Simulate matchups and outcomes.
slice_tournament_matchups, slice_tournament_outcomes = game_predictions.simulate_tournament(tournament_bracket, pre_tournament_stats, features, deterministic=True, coef_trace=slice_trace)

In [81]:
slice_tournament_matchups.head()

,winner,loser,round_of,count
0,Arizona,Baylor,16,9998
1,Arizona,Gonzaga,2,2
2,Arizona,Iowa St.,2,1
3,Arizona,Kentucky,4,9
4,Arizona,Ohio St.,32,6941


In [82]:
slice_tournament_matchups[slice_tournament_matchups.round_of == 2].sort('count', ascending=False).head()

,winner,loser,round_of,count
82,Kentucky,Villanova,2,2945
73,Kentucky,Gonzaga,2,1693
72,Kentucky,Duke,2,1558
83,Kentucky,Virginia,2,1013
190,Wisconsin,Villanova,2,829


In [83]:
slice_tournament_outcomes

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Albany,0,0,0,0,0,0
Arizona,10000,10000,10000,3216,9,5
Arkansas,10000,4,0,0,0,0
Baylor,10000,9998,0,0,0,0
Belmont,0,0,0,0,0,0
Buffalo,92,40,0,0,0,0
Butler,4850,29,7,0,0,0
Cincinnati,5034,0,0,0,0,0
Coastal Carolina,0,0,0,0,0,0
Davidson,5541,0,0,0,0,0


In [84]:
slice_tournament_outcomes.sort('wins_round_6', ascending=False).head(n=7)

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Kentucky,10000,10000,10000,9988,7343,7210
Wisconsin,10000,10000,10000,6784,2648,2173
Villanova,10000,10000,10000,7926,4356,581
Duke,10000,9994,9409,4803,2256,22
Virginia,10000,9914,9914,2074,1110,7
Arizona,10000,10000,10000,3216,9,5
Gonzaga,10000,10000,9772,4976,2276,2


#### Final Four

In [85]:
# Simulate matchups and outcomes.
slice_final_four_matchups, slice_final_four_outcomes = game_predictions.simulate_tournament(final_four_bracket, pre_tournament_stats, features, deterministic=True, coef_trace=slice_trace)

In [87]:
slice_final_four_matchups

,winner,loser,round_of,count
0,Duke,Kentucky,2,42
1,Duke,Michigan St.,4,10000
2,Duke,Wisconsin,2,8
3,Kentucky,Duke,2,6315
4,Kentucky,Wisconsin,4,6357
5,Wisconsin,Duke,2,3635
6,Wisconsin,Kentucky,4,3643


In [88]:
slice_final_four_outcomes

,wins_round_1,wins_round_2
Duke,10000,50
Kentucky,6357,6315
Michigan St.,0,0
Wisconsin,3643,3635


### Metropolis-Hastings

#### Whole-Tournament

In [56]:
# Simulate matchups and outcomes.
mh_tournament_matchups, mh_tournament_outcomes = game_predictions.simulate_tournament(tournament_bracket, pre_tournament_stats, features, deterministic=True, coef_trace=mh_trace)

In [68]:
mh_tournament_matchups.head()

,winner,loser,round_of,count
0,Arizona,Baylor,16,9973
1,Arizona,Duke,2,43
2,Arizona,Gonzaga,2,70
3,Arizona,Iowa St.,2,2
4,Arizona,Kentucky,4,435


In [69]:
mh_tournament_matchups[mh_tournament_matchups.round_of == 2].sort('count', ascending=False).head()

,winner,loser,round_of,count
92,Kentucky,Villanova,2,2433
93,Kentucky,Virginia,2,2242
82,Kentucky,Gonzaga,2,1946
81,Kentucky,Duke,2,1212
200,Wisconsin,Gonzaga,2,321


In [70]:
mh_tournament_outcomes

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Albany,0,0,0,0,0,0
Arizona,10000,10000,9999,4826,435,246
Arkansas,10000,27,0,0,0,0
Baylor,10000,9974,1,0,0,0
Belmont,0,0,0,0,0,0
Buffalo,63,34,0,0,0,0
Butler,4975,258,72,0,0,0
Cincinnati,5391,0,0,0,0,0
Coastal Carolina,0,0,0,0,0,0
Davidson,4651,1,1,0,0,0


In [72]:
mh_tournament_outcomes.sort('wins_round_6', ascending=False).head(n=7)

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Kentucky,10000,10000,10000,9977,8035,7862
Wisconsin,10000,10000,10000,5174,1529,1184
Villanova,10000,10000,10000,5562,3245,481
Arizona,10000,10000,9999,4826,435,246
Virginia,10000,9966,9966,4438,2698,126
Duke,10000,9934,6870,3594,1637,65
Gonzaga,10000,9999,9583,4879,2373,36


#### Final Four

In [52]:
# Simulate matchups and outcomes.
mh_final_four_matchups, mh_final_four_outcomes = game_predictions.simulate_tournament(final_four_bracket, pre_tournament_stats, features, deterministic=True, coef_trace=mh_trace)

In [75]:
mh_final_four_matchups

,winner,loser,round_of,count
0,Duke,Kentucky,2,46
1,Duke,Michigan St.,4,10000
2,Duke,Wisconsin,2,185
3,Kentucky,Duke,2,7118
4,Kentucky,Wisconsin,4,7164
5,Wisconsin,Duke,2,2651
6,Wisconsin,Kentucky,4,2836


In [76]:
mh_final_four_outcomes

,wins_round_1,wins_round_2
Duke,10000,231
Kentucky,7164,7118
Michigan St.,0,0
Wisconsin,2836,2651


---

## Monte Carlo Simulations

### Slice

#### Whole Tournament

In [30]:
# Simulate matchups and outcomes.
slice_mc_tournament_matchups, slice_mc_tournament_outcomes = game_predictions.simulate_tournament(tournament_bracket, pre_tournament_stats, features, deterministic=False, coef_trace=slice_trace)

In [45]:
slice_mc_tournament_matchups.head()

,winner,loser,round_of,count
0,Albany,Belmont,16,1
1,Albany,Dayton,32,76
2,Albany,Georgia,16,1
3,Albany,Michigan St.,16,1
4,Albany,Oklahoma,64,933


In [33]:
slice_mc_tournament_matchups[slice_mc_tournament_matchups.round_of == 2].sort('count', ascending=False).head()

,winner,loser,round_of,count
487,Kentucky,Villanova,2,1083
452,Kentucky,Duke,2,748
456,Kentucky,Gonzaga,2,575
1087,Villanova,Kentucky,2,575
488,Kentucky,Virginia,2,544


In [34]:
slice_mc_tournament_outcomes

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Albany,933,164,10,0,0,0
Arizona,9882,8625,7082,3491,1489,874
Arkansas,8146,3163,427,89,15,3
Baylor,8261,5626,1627,368,77,27
Belmont,406,72,11,0,0,0
Buffalo,3614,1840,101,16,0,0
Butler,4504,1131,410,44,0,0
Cincinnati,5437,230,114,34,2,1
Coastal Carolina,155,19,0,0,0,0
Davidson,5393,1234,499,117,32,9


In [35]:
slice_mc_tournament_outcomes.sort('wins_round_6', ascending=False).head(n=7)

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Kentucky,9975,9614,9104,7743,4971,3613
Wisconsin,9845,9158,7947,5337,2726,1808
Villanova,9837,8968,7371,5224,3379,1438
Arizona,9882,8625,7082,3491,1489,874
Duke,9818,8802,6587,4016,2097,706
Gonzaga,9763,7829,5584,3122,1544,507
Virginia,9594,7505,5740,2710,1418,463


##### Export Data

In [97]:
# Reformat matchups.
slice_mc_tournament_matchups_reformatted = reformat_matchups(slice_mc_tournament_matchups)

# Dump data.
slice_mc_tournament_matchups_reformatted.to_csv(export_base_path+'tournament/slice_matchups.csv', index=False)
slice_mc_tournament_outcomes.reset_index().rename(columns={'index':'team'}).to_csv(export_base_path+'tournament/slice_outcomes.csv', index=False)

#### Final Four

In [19]:
# Simulate matchups and outcomes.
slice_mc_final_four_matchups, slice_mc_final_four_outcomes = game_predictions.simulate_tournament(final_four_bracket, pre_tournament_stats, features, deterministic=False, coef_trace=slice_trace)

In [20]:
slice_mc_final_four_matchups

,winner,loser,round_of,count
0,Duke,Kentucky,2,1300
1,Duke,Michigan St.,4,7757
2,Duke,Wisconsin,2,1059
3,Kentucky,Duke,2,3269
4,Kentucky,Michigan St.,2,1195
5,Kentucky,Wisconsin,4,5871
6,Michigan St.,Duke,4,2243
7,Michigan St.,Kentucky,2,107
8,Michigan St.,Wisconsin,2,127
9,Wisconsin,Duke,2,2129


In [21]:
slice_mc_final_four_outcomes

,wins_round_1,wins_round_2
Duke,7757,2359
Kentucky,5871,4464
Michigan St.,2243,234
Wisconsin,4129,2943


##### Export Data

In [98]:
# Reformat matchups.
slice_mc_final_four_matchups_reformatted = reformat_matchups(slice_mc_final_four_matchups)

# Dump data.
slice_mc_final_four_matchups_reformatted.to_csv(export_base_path+'final_four/slice_matchups.csv', index=False)
slice_mc_final_four_outcomes.reset_index().rename(columns={'index':'team'}).to_csv(export_base_path+'final_four/slice_outcomes.csv', index=False)

#### Championship

In [28]:
# Simulate matchups and outcomes.
_, slice_mc_championship_outcomes = game_predictions.simulate_tournament(championship_bracket, pre_tournament_stats, features, deterministic=False, coef_trace=slice_trace)

In [29]:
slice_mc_championship_outcomes

,wins_round_1
Duke,3360
Wisconsin,6640


In [99]:
# Export.
slice_mc_championship_outcomes.reset_index().rename(columns={'index':'team'}).to_csv(export_base_path+'championship/slice_outcomes.csv', index=False)

### Metropolis-Hastings

#### Whole Tournament

In [31]:
# Simulate matchups and outcomes.
mh_mc_tournament_matchups, mh_mc_tournament_outcomes = game_predictions.simulate_tournament(tournament_bracket, pre_tournament_stats, features, deterministic=False, coef_trace=mh_trace)

In [44]:
mh_mc_tournament_matchups.head()

,winner,loser,round_of,count
0,Albany,Belmont,16,1
1,Albany,Dayton,32,77
2,Albany,Georgia,16,1
3,Albany,Michigan St.,16,6
4,Albany,Oklahoma,64,878


In [37]:
mh_mc_tournament_matchups[mh_mc_tournament_matchups.round_of == 2].sort('count', ascending=False).head()

,winner,loser,round_of,count
586,Kentucky,Villanova,2,832
587,Kentucky,Virginia,2,583
553,Kentucky,Gonzaga,2,519
549,Kentucky,Duke,2,506
1303,Villanova,Kentucky,2,422


In [38]:
mh_mc_tournament_outcomes

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Albany,878,151,11,0,0,0
Arizona,9873,8251,6663,3846,1872,1135
Arkansas,7709,3227,659,168,34,6
Baylor,8022,5307,1635,535,136,53
Belmont,356,54,13,1,0,0
Buffalo,3489,1639,149,37,7,1
Butler,4636,1673,644,112,17,6
Cincinnati,5766,524,277,85,14,4
Coastal Carolina,237,36,10,1,1,0
Davidson,4655,1136,405,127,31,5


In [39]:
mh_mc_tournament_outcomes.sort('wins_round_6', ascending=False).head(n=7)

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Kentucky,9946,9187,8442,6990,4598,3327
Wisconsin,9763,8729,7112,4369,2222,1411
Arizona,9873,8251,6663,3846,1872,1135
Villanova,9830,8599,6565,4207,2671,1131
Virginia,9644,7472,5489,3060,1843,757
Duke,9762,7868,5372,3129,1578,596
Gonzaga,9635,7509,5179,3108,1575,573


##### Export Data

In [ ]:
dd

In [100]:
# Reformat matchups.
mh_mc_tournament_matchups_reformatted = reformat_matchups(mh_mc_tournament_matchups)

# Dump data.
mh_mc_tournament_matchups_reformatted.to_csv(export_base_path+'tournament/mh_matchups.csv', index=False)
mh_mc_tournament_outcomes.reset_index().rename(columns={'index':'team'}).to_csv(export_base_path+'tournament/mh_outcomes.csv', index=False)

#### Final Four

In [16]:
# Simulate matchups and outcomes.
mh_mc_final_four_matchups, mh_mc_final_four_outcomes = game_predictions.simulate_tournament(final_four_bracket, pre_tournament_stats, features, deterministic=False, coef_trace=mh_trace)

In [17]:
mh_mc_final_four_matchups

,winner,loser,round_of,count
0,Duke,Kentucky,2,1189
1,Duke,Michigan St.,4,7201
2,Duke,Wisconsin,2,1087
3,Kentucky,Duke,2,3069
4,Kentucky,Michigan St.,2,1464
5,Kentucky,Wisconsin,4,5904
6,Michigan St.,Duke,4,2799
7,Michigan St.,Kentucky,2,182
8,Michigan St.,Wisconsin,2,215
9,Wisconsin,Duke,2,1856


In [18]:
mh_mc_final_four_outcomes

,wins_round_1,wins_round_2
Duke,7201,2276
Kentucky,5904,4533
Michigan St.,2799,397
Wisconsin,4096,2794


##### Export Data

In [101]:
# Reformat matchups.
mh_mc_final_four_matchups_reformatted = reformat_matchups(mh_mc_final_four_matchups)

# Dump data.
mh_mc_final_four_matchups_reformatted.to_csv(export_base_path+'final_four/mh_matchups.csv', index=False)
mh_mc_final_four_outcomes.reset_index().rename(columns={'index':'team'}).to_csv(export_base_path+'final_four/mh_outcomes.csv', index=False)

#### Championship

In [25]:
# Simulate matchups and outcomes.
_, mh_mc_championship_outcomes = game_predictions.simulate_tournament(championship_bracket, pre_tournament_stats, features, deterministic=False, coef_trace=mh_trace)

In [27]:
mh_mc_championship_outcomes

,wins_round_1
Duke,3633
Wisconsin,6367


In [102]:
# Export.
mh_mc_championship_outcomes.reset_index().rename(columns={'index':'team'}).to_csv(export_base_path+'championship/mh_outcomes.csv', index=False)

### Aggregate

#### Tournament

In [122]:
# Aggregate matchups.
agg_mc_tournament_matchups = slice_mc_tournament_matchups_reformatted.append(mh_mc_tournament_matchups_reformatted)
agg_mc_tournament_matchups = agg_mc_tournament_matchups.groupby(['team_1','team_2','round_of']).agg(np.sum).reset_index()
# Export.
agg_mc_tournament_matchups.to_csv(export_base_path+'tournament/agg_matchups.csv', index=False)

In [110]:
# Aggregate outcomes.
agg_mc_tournament_outcomes = slice_mc_tournament_outcomes.append(mh_mc_tournament_outcomes)
agg_mc_tournament_outcomes = agg_mc_tournament_outcomes.groupby(agg_mc_tournament_outcomes.index).agg(np.sum)
# Export.
agg_mc_tournament_outcomes.reset_index().rename(columns={'index':'team'}).to_csv(export_base_path+'tournament/agg_outcomes.csv', index=False)

In [129]:
agg_mc_tournament_matchups[agg_mc_tournament_matchups.round_of == 2].sort('team_1_wins',ascending=False).head()

,team_1,team_2,round_of,team_1_wins,team_2_wins
582,Kentucky,Villanova,2,1915,997
583,Kentucky,Virginia,2,1127,440
925,Villanova,Wisconsin,2,652,891
56,Arizona,Villanova,2,494,501
293,Duke,Kentucky,2,480,1254


In [132]:
agg_mc_tournament_outcomes.sort('wins_round_6',ascending=False).head(n=10)

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Kentucky,19921,18801,17546,14733,9569,6940
Wisconsin,19608,17887,15059,9706,4948,3219
Villanova,19667,17567,13936,9431,6050,2569
Arizona,19755,16876,13745,7337,3361,2009
Duke,19580,16670,11959,7145,3675,1302
Virginia,19238,14977,11229,5770,3261,1220
Gonzaga,19398,15338,10763,6230,3119,1080
Notre Dame,18644,13419,8603,2376,799,354
Utah,14854,10611,4475,2140,843,211
Iowa St.,18482,12566,5334,2357,875,199


In [133]:
agg_mc_tournament_outcomes[agg_mc_tournament_outcomes.index == 'Harvard']

,wins_round_1,wins_round_2,wins_round_3,wins_round_4,wins_round_5,wins_round_6
Harvard,2751,854,57,4,0,0


#### Final Four

In [124]:
# Aggregate matchups.
agg_mc_final_four_matchups = slice_mc_final_four_matchups_reformatted.append(mh_mc_final_four_matchups_reformatted)
agg_mc_final_four_matchups = agg_mc_final_four_matchups.groupby(['team_1','team_2','round_of']).agg(np.sum).reset_index()
# Export.
agg_mc_final_four_matchups.to_csv(export_base_path+'final_four/agg_matchups.csv', index=False)

In [111]:
# Aggregate outcomes.
agg_mc_final_four_outcomes = slice_mc_final_four_outcomes.append(mh_mc_final_four_outcomes)
agg_mc_final_four_outcomes = agg_mc_final_four_outcomes.groupby(agg_mc_final_four_outcomes.index).agg(np.sum)
# Export.
agg_mc_final_four_outcomes.reset_index().rename(columns={'index':'team'}).to_csv(export_base_path+'final_four/agg_outcomes.csv', index=False)

In [135]:
agg_mc_final_four_outcomes.sort('wins_round_2',ascending=False).head(n=10)

,wins_round_1,wins_round_2
Kentucky,11775,8997
Wisconsin,8225,5737
Duke,14958,4635
Michigan St.,5042,631


#### Championship

In [112]:
# Aggregate outcomes.
agg_mc_championship_outcomes = slice_mc_championship_outcomes.append(mh_mc_championship_outcomes)
agg_mc_championship_outcomes = agg_mc_championship_outcomes.groupby(agg_mc_championship_outcomes.index).agg(np.sum)
# Export.
agg_mc_championship_outcomes.reset_index().rename(columns={'index':'team'}).to_csv(export_base_path+'championship/agg_outcomes.csv', index=False)